In [1]:
import findspark
findspark.init()
findspark.find()

'C:\\work\\spark-2.4.8-bin-hadoop2.7'

In [3]:
from pyspark.sql import SparkSession
spark=SparkSession.builder.appName('Temp').getOrCreate()

In [22]:
from pyspark.sql.types import *
from pyspark.sql import functions as func

In [5]:

schema=StructType([
    StructField("stationID",StringType(),True),
    StructField("date",IntegerType(),True),
    StructField("measure-type",StringType(),True),
    StructField("temperature",FloatType(),True)
])

In [6]:
data=spark.read.schema(schema).csv("1800.csv")
data.show(5)

+-----------+--------+------------+-----------+
|  stationID|    date|measure-type|temperature|
+-----------+--------+------------+-----------+
|ITE00100554|18000101|        TMAX|      -75.0|
|ITE00100554|18000101|        TMIN|     -148.0|
|GM000010962|18000101|        PRCP|        0.0|
|EZE00100082|18000101|        TMAX|      -86.0|
|EZE00100082|18000101|        TMIN|     -135.0|
+-----------+--------+------------+-----------+
only showing top 5 rows



In [17]:
df=data.filter(data["measure-type"]=="TMIN")

In [18]:
df=df.select("stationID","temperature")

In [20]:
df=df.groupBy("stationID").min("temperature") \
.withColumnRenamed("min(temperature)", "temperature")
df.show()

+-----------+-----------+
|  stationID|temperature|
+-----------+-----------+
|ITE00100554|     -148.0|
|EZE00100082|     -135.0|
+-----------+-----------+



In [23]:
minTempF=df.withColumn("temperatureF",
                      func.round(func.col("temperature")*(9.0/5.0)+32.0)) \
                    .select("stationID","temperatureF").sort("temperatureF")
minTempF.show()

+-----------+------------+
|  stationID|temperatureF|
+-----------+------------+
|ITE00100554|      -234.0|
|EZE00100082|      -211.0|
+-----------+------------+

